<a href="https://colab.research.google.com/github/hykim-1/Study/blob/main/weather_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -----------------------------
# Weather - Regression (회귀 + MSE 등)
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 1) 데이터 로드
df = pd.read_csv(
    "/content/drive/MyDrive/weather.csv",
    na_values=["NA"],
    parse_dates=["Date"],
)
df = df.set_index("Date")

# 2) 누설/불필요 컬럼 제거
for col in ["RainTomorrow", "RISK_MM", "WeatherTomorrow", "weather_tomorrow", "내일날씨", "Tomorrow"]:
    if col in df.columns:
        df = df.drop(columns=[col])

# 3) 결측치 제거
df = df.dropna()

# 4) 범주형 → 라벨인코딩
label_encoders = {}
for col in df.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# 5) X, y 분리 (타깃: Rainfall)
y = df["Rainfall"]
X = df.drop(columns=["Rainfall"])

# 6) train/test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=12
)

# 7) 파이프라인 (X만 스케일링 → 선형회귀)
pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

# 8) 학습 & 예측
pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

# 9) 평가 지표
mse  = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, pred)
r2   = r2_score(y_test, pred)

print(f"MSE : {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")
print(f"R^2 : {r2:.4f}")


MSE : 22.9130
RMSE: 4.7868
MAE : 2.0898
R^2 : 0.2894
